In [3]:
from dataretrieval import nwis
from dataretrieval import wqp
import pandas as pd

# NWM dataset 
# https://github.com/NOAA-OWP/hydrotools?tab=readme-ov-file


### Data source, availability, and information
From upstream to downsteam
#### 1. Skagit River Above Miller Creek Near Rockport, WA - 12189700
source: https://waterdata.usgs.gov/monitoring-location/12189700/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false\
Availability: Gage height, feet (from 5/2020).\
Latitude 48°28'50.80",   Longitude 121°39'04.70"   NAD83.\
Skagit County, Washington, Hydrologic Unit 17110005.\
Datum of gage: 199.93 feet above   NAVD88.\


#### 2. Lake Shannon at Concrete, WA - 12193000 
source: https://waterdata.usgs.gov/monitoring-location/12193000/#dataTypeId=continuous-62615-0&period=P7D&showMedian=true\
Availability: Gage height, feet (from 5/2020)\
Latitude 48°32'53",   Longitude 121°44'22"   NAD27\
Skagit County, Washington, Hydrologic Unit 17110005\
Drainage area: 297 square miles\
Datum of gage: 0. feet above   NAVD88.\
Note: Location inside the dam\

#### 3. Baker River at Henry Thompson BR at Concrete, WA - 12193400 
Note: inflow discharge B\
source: https://waterdata.usgs.gov/monitoring-location/12193400/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false\
Availability: Gage height, discharge\
Latitude 48°32'26",   Longitude 121°44'32"   NAD27\
Skagit County, Washington, Hydrologic Unit 17110005\
Drainage area: 297 square miles\
Datum of gage: 174.08 feet above   NAVD88.

#### 4. Skagit River Near Concrete, WA - 12194000
Note: inflow discharge C\
source: https://waterdata.usgs.gov/monitoring-location/12194000/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false\
Availability: Gage height, discharge\
Latitude 48°31'28",   Longitude 121°46'11"   NAD27\
Skagit County, Washington, Hydrologic Unit 17110007\
Drainage area: 2,737 square miles\
Datum of gage: 133.96 feet above   NAVD88.

#### 5. Finney Creek at South Skagit Hwy NR Concrete, WA - 12194580
source: https://waterdata.usgs.gov/monitoring-location/12194580/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false\
Availability: Gage height, data from 2024-05-23\
Latitude 48°31'17.91",   Longitude 121°50'24.04"   NAD83\
Skagit County, Washington, Hydrologic Unit 17110007\
Datum of gage: 133 feet above   NAVD88.

#### 6. Skagit River Near Sedro Woolley, WA - 12199000
source: https://waterdata.usgs.gov/monitoring-location/12199000/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false\
Availability: Gage height \
Latitude 48°29'03",   Longitude 122°14'31"   NAD27\
Skagit County, Washington, Hydrologic Unit 17110007\
Drainage area: 3,015 square miles\
Datum of gage: 3.81 feet above   NAVD88.

#### 7. Skagit River Near Mount Vernon, WA - 12200500
source: https://waterdata.usgs.gov/monitoring-location/12200500/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false\
Availability: Gage height, discharge, temperature, turbidity (9/2016 - 10/2017) \
Latitude 48°26'42",   Longitude 122°20'03"   NAD27\
Skagit County, Washington, Hydrologic Unit 17110007\
Drainage area: 3,093 square miles\
Datum of gage: 3.80 feet above   NAVD88.

#### 8. Sauk River Near Sauk, WA - 12189500
Note: Can be used as temperature input
source: https://waterdata.usgs.gov/monitoring-location/12189500/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false\
Availability: Gage height, discharge, temperature, turbidity (9/2011 - 4/2021) \
Latitude 48°25'29",   Longitude 121°34'02"   NAD27\
Skagit County, Washington, Hydrologic Unit 17110006\
Drainage area: 714 square miles\
Datum of gage: 266 feet above   NGVD29.

#### 9. Skagit River at Marblemount, WA - 12181000
Note: Can be used as temperature input\
source: https://waterdata.usgs.gov/monitoring-location/12181000/#dataTypeId=continuous-00065-85281465&period=P7D&showMedian=false\
Availability: Gage height, discharge, temperature\
Latitude 48°32'02",   Longitude 121°25'43"   NAD27\
Skagit County, Washington, Hydrologic Unit 17110005\
Drainage area: 1,381 square miles\
Datum of gage: 308.64 feet above   NAVD88.

In [4]:
# site numbers for the USGS gage
siteNumbers = ["12189700","12193000", "12193400", "12194000","12194580","12199000","12200500","12189500","12181000"]
#siteNumbers = ["12193400"]

parameterCds = ["00065","00060","00010","00062"]  # wse, discharge, and temperature, wse above datum
#statCd = ["00001","00003"]  # Mean and maximum
startDate = "2021-01-01"
endDate = "2023-01-02"


for site in siteNumbers:
    
    # Get site information
    chop_tank_info, md = nwis.get_info(sites=site)

    # print site information
    print(f"Site information for {site}:")
    csv_filename = f"info_site_{site}.csv"
    chop_tank_info.to_csv(csv_filename)

    #rawData, md = nwis.get_dv(sites=siteNumbers, parameterCd=parameterCds,
    #                                     start=startDate, end=endDate)
    rawData, md = nwis.get_iv(sites=site, parameterCd=parameterCds,
                            start=startDate, end=endDate)
     
    # Check if columns exist before renaming
    rename_mapping = {
        "00060": "Discharge (cfs)",
        "00065": "Gage Height (ft)",
        "00010": "Temperature (C)",
        "00062": "WSE above datum (ft)",
    }

    # Check if the columns to rename exist in the DataFrame
    # and rename them accordingly 
    existing_columns = rawData.columns.intersection(rename_mapping.keys())
    if not existing_columns.empty:
        rawData = rawData.rename(columns={col: rename_mapping[col] for col in existing_columns})
        # Drop columns ending with '_cd'
        columns_to_drop = [col + '_cd' for col in existing_columns]
        rawData = rawData.drop(columns=columns_to_drop)
    
    # Remove timezone from datetime and convert to local time
    rawData=rawData.tz_localize(None)
    rawData.index=rawData.index-pd.Timedelta(hours=8)
    
    # Convert index to a column
    rawData = rawData.reset_index().rename(columns={'datetime': 'date'})
    
    # Save rawData to a CSV file
    csv_filename = f"rawData_site_{site}.csv"
    rawData.to_csv(csv_filename, index=False)
    print(f"Data for site {site} saved to {csv_filename}")
    print(rawData.head(10))

Site information for 12189700:
Data for site 12189700 saved to rawData_site_12189700.csv
                 date   site_no  Gage Height (ft)
0 2021-01-01 00:00:00  12189700              3.34
1 2021-01-01 00:15:00  12189700              3.33
2 2021-01-01 00:30:00  12189700              3.32
3 2021-01-01 00:45:00  12189700              3.32
4 2021-01-01 01:00:00  12189700              3.31
5 2021-01-01 01:15:00  12189700              3.31
6 2021-01-01 01:30:00  12189700              3.30
7 2021-01-01 01:45:00  12189700              3.30
8 2021-01-01 02:00:00  12189700              3.29
9 2021-01-01 02:15:00  12189700              3.29
Site information for 12193000:
Data for site 12193000 saved to rawData_site_12193000.csv
Empty DataFrame
Columns: [date, site_no]
Index: []
Site information for 12193400:
Data for site 12193400 saved to rawData_site_12193400.csv
                 date   site_no  Discharge (cfs)  Gage Height (ft)
0 2021-01-01 00:00:00  12193400           1680.0              3.1

In [5]:
# Print the first 10 rows of the DataFrame
headRawData = rawData.head(10)
parameter_codes = nwis.get_pmcodes('All')
print(parameter_codes)
print("Retrieved information about " + str(len(parameter_codes[0])) + " parameter code.")

(      parm_cd        group                                            parm_nm  \
0       00001  Information  Location in cross section, distance from right...   
1       00002  Information  Location in cross section, distance from right...   
2       00003  Information                               Sampling depth, feet   
3       00004     Physical                                 Stream width, feet   
4       00005  Information  Location in cross section, fraction of total d...   
...       ...          ...                                                ...   
19026   99994  Information  Diazinon-d10, surrogate, NWQL schedules 2003/2...   
19027   99995  Information  alpha-HCH-d6, surrogate, NWQL schedules 2003/2...   
19028   99996  Information  Sample weight, NWQL schedule OCSS, suspended s...   
19029   99997     Physical  Photosynthetically active radiation (PAR) in w...   
19030   99998     Physical  Photosynthetically active radiation (PAR), wat...   

                  epa_equi